In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv1D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, MaxPooling1D, Concatenate,Dropout, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, UpSampling1D
from keras.regularizers import l2, l1, l1_l2

# The Speck cipher and data generation algorithms #
Taken from https://github.com/agohr/deep_speck/blob/master/speck.py

In [ ]:
import numpy as np
from os import urandom

def WORD_SIZE():
    return(16);

def ALPHA():
    return(7);

def BETA():
    return(2);

MASK_VAL = 2 ** WORD_SIZE() - 1;

def shuffle_together(l):
    state = np.random.get_state();
    for x in l:
        np.random.set_state(state);
        np.random.shuffle(x);

def rol(x,k):
    return(((x << k) & MASK_VAL) | (x >> (WORD_SIZE() - k)));

def ror(x,k):
    return((x >> k) | ((x << (WORD_SIZE() - k)) & MASK_VAL));

def enc_one_round(p, k):
    c0, c1 = p[0], p[1];
    c0 = ror(c0, ALPHA());
    c0 = (c0 + c1) & MASK_VAL;
    c0 = c0 ^ k;
    c1 = rol(c1, BETA());
    c1 = c1 ^ c0;
    return(c0,c1);

def dec_one_round(c,k):
    c0, c1 = c[0], c[1];
    c1 = c1 ^ c0;
    c1 = ror(c1, BETA());
    c0 = c0 ^ k;
    c0 = (c0 - c1) & MASK_VAL;
    c0 = rol(c0, ALPHA());
    return(c0, c1);

def expand_key(k, t):
    ks = [0 for i in range(t)];
    ks[0] = k[len(k)-1];
    l = list(reversed(k[:len(k)-1]));
    for i in range(t-1):
        l[i%3], ks[i+1] = enc_one_round((l[i%3], ks[i]), i);
    return(ks);

def encrypt(p, ks):
    x, y = p[0], p[1];
    for k in ks:
        x,y = enc_one_round((x,y), k);
    return(x, y);

def decrypt(c, ks):
    x, y = c[0], c[1];
    for k in reversed(ks):
        x, y = dec_one_round((x,y), k);
    return(x,y);

def check_testvector():
  key = (0x1918,0x1110,0x0908,0x0100)
  pt = (0x6574, 0x694c)
  ks = expand_key(key, 22)
  ct = encrypt(pt, ks)
  if (ct == (0xa868, 0x42f2)):
    print("Testvector verified.")
    return(True);
  else:
    print("Testvector not verified.")
    return(False);

#convert_to_binary takes as input an array of ciphertext pairs
#where the first row of the array contains the lefthand side of the ciphertexts,
#the second row contains the righthand side of the ciphertexts,
#the third row contains the lefthand side of the second ciphertexts,
#and so on
#it returns an array of bit vectors containing the same data
def convert_to_binary(arr):
  X = np.zeros((4 * WORD_SIZE(),len(arr[0])),dtype=np.uint8);
  for i in range(4 * WORD_SIZE()):
    index = i // WORD_SIZE();
    offset = WORD_SIZE() - (i % WORD_SIZE()) - 1;
    X[i] = (arr[index] >> offset) & 1;
  X = X.transpose();
  return(X);

#takes a text file that contains encrypted block0, block1, true diff prob, real or random
#data samples are line separated, the above items whitespace-separated
#returns train data, ground truth, optimal ddt prediction
def readcsv(datei):
    data = np.genfromtxt(datei, delimiter=' ', converters={x: lambda s: int(s,16) for x in range(2)});
    X0 = [data[i][0] for i in range(len(data))];
    X1 = [data[i][1] for i in range(len(data))];
    Y = [data[i][3] for i in range(len(data))];
    Z = [data[i][2] for i in range(len(data))];
    ct0a = [X0[i] >> 16 for i in range(len(data))];
    ct1a = [X0[i] & MASK_VAL for i in range(len(data))];
    ct0b = [X1[i] >> 16 for i in range(len(data))];
    ct1b = [X1[i] & MASK_VAL for i in range(len(data))];
    ct0a = np.array(ct0a, dtype=np.uint16); ct1a = np.array(ct1a,dtype=np.uint16);
    ct0b = np.array(ct0b, dtype=np.uint16); ct1b = np.array(ct1b, dtype=np.uint16);
    
    #X = [[X0[i] >> 16, X0[i] & 0xffff, X1[i] >> 16, X1[i] & 0xffff] for i in range(len(data))];
    X = convert_to_binary([ct0a, ct1a, ct0b, ct1b]); 
    Y = np.array(Y, dtype=np.uint8); Z = np.array(Z);
    return(X,Y,Z);

#baseline training data generator
def make_train_data(n, nr, diff=(0x0040,0)):
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  plain1l[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  plain1r[Y==0] = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);

#real differences data generator
def real_differences_data(n, nr, diff=(0x0040,0)):
  #generate labels
  Y = np.frombuffer(urandom(n), dtype=np.uint8); Y = Y & 1;
  #generate keys
  keys = np.frombuffer(urandom(8*n),dtype=np.uint16).reshape(4,-1);
  #generate plaintexts
  plain0l = np.frombuffer(urandom(2*n),dtype=np.uint16);
  plain0r = np.frombuffer(urandom(2*n),dtype=np.uint16);
  #apply input difference
  plain1l = plain0l ^ diff[0]; plain1r = plain0r ^ diff[1];
  num_rand_samples = np.sum(Y==0);
  #expand keys and encrypt
  ks = expand_key(keys, nr);
  ctdata0l, ctdata0r = encrypt((plain0l, plain0r), ks);
  ctdata1l, ctdata1r = encrypt((plain1l, plain1r), ks);
  #generate blinding values
  k0 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  k1 = np.frombuffer(urandom(2*num_rand_samples),dtype=np.uint16);
  #apply blinding to the samples labelled as random
  ctdata0l[Y==0] = ctdata0l[Y==0] ^ k0; ctdata0r[Y==0] = ctdata0r[Y==0] ^ k1;
  ctdata1l[Y==0] = ctdata1l[Y==0] ^ k0; ctdata1r[Y==0] = ctdata1r[Y==0] ^ k1;
  #convert to input data for neural networks
  X = convert_to_binary([ctdata0l, ctdata0r, ctdata1l, ctdata1r]);
  return(X,Y);


# Evaluate the results #
Taken from https://github.com/agohr/deep_speck/blob/master/eval.py and slightly adapted.

In [ ]:
def evaluate(net,X,Y):
    Z = net.predict(X,batch_size=5000).flatten();
    Zbin = (Z > 0.5);
    diff = Y.flatten() - Z; 
    mse = np.mean(diff*diff);
    n = len(Z); n0 = np.sum(Y==0); n1 = np.sum(Y==1);
    acc = np.sum(Zbin == Y.flatten()) / n;
    tpr = np.sum(Zbin[Y.flatten()==1]) / n1;
    tnr = np.sum(Zbin[Y.flatten()==0] == 0) / n0;
    mreal = np.median(Z[Y.flatten()==1]);
    high_random = np.sum(Z[Y.flatten()==0] > mreal) / n0;

    return(acc, tpr, tnr); # added
    #print("Accuracy: ", acc, "TPR: ", tpr, "TNR: ", tnr, "MSE:", mse);
    #print("Percentage of random pairs with score higher than median of real pairs:", 100*high_random);

# Conducting multiple evaluations #


In [ ]:
def multiple_evaluations(model, repetitions, num_rounds):
 
  accs = [];
  tprs = [];
  tnrs = [];
  for i in range(0, repetitions):
    X_eval, Y_eval = make_train_data(10**6, num_rounds);
    (acc, tpr, tnr) = evaluate(model, X_eval, X_eval);
    accs.append(acc);
    tprs.append(tpr);
    tnrs.append(tnr);

  print("Acc: " + str(np.mean(accs)) + str(" +- ") + str(np.std(accs)) + str("\t") + 
        "Tpr:" + str(np.mean(tprs)) + str(" +- ") + str(np.std(tprs)) + str("\t") +
        "Tnr:" + str(np.mean(tnrs)) + str(" +- ") + str(np.std(tnrs)) + str("\t"));
        
  return(np.mean(accs), np.mean(tprs), np.mean(tnrs));

# Building the Autoencoder #
The implementation for all three layers is provided below

In [ ]:

def encode(input):

  x = Reshape((4, 16))(input);
  x = Permute((2,1))(x);

  #Layer 1
  x = Conv1D(32, 3, padding='same')(x)
  x = BatchNormalization()(x);
  x = Activation('relu')(x);
  x = MaxPooling1D(2, padding='same')(x)

  #Layer 2
  #x = Conv1D(32, 3, padding='same')(x)
  #x = BatchNormalization()(x);
  #x = Activation('relu')(x);
  #x = MaxPooling1D(2, padding='same')(x)
    
  #Layer 3  
  #x = Conv1D(32, 3, padding='same')(x)
  #x = BatchNormalization()(x);
  #x = Activation('relu')(x);
  #x = MaxPooling1D(2, padding='same')(x)

  encoded = x;
    


  return encoded;


def decode(encoded):
  
  x = Conv1D(32, 3, padding='same')(encoded)
  x = BatchNormalization()(x);
  x = Activation('relu')(x);
  x = UpSampling1D(2)(x)

  #x = Conv1D(32, 3, padding='same')(x)
  #x = BatchNormalization()(x);
  #x = Activation('relu')(x);
  #x = UpSampling1D(2)(x)

  #x = Conv1D(32, 3, padding='same')(x)
  #x = BatchNormalization()(x);
  #x = Activation('relu')(x);
  #x = UpSampling1D(2)(x)

 
  x = Conv1D(4, 3, activation='sigmoid', padding='same')(x)
  x = Permute((2,1))(x);
  decoded = Reshape((64,1))(x);

  return decoded;


def build_autoencoder():
  inp = Input(shape=(64,));
  encoded= encode(inp);
  decoded = decode(encoded);
  autoencoder = Model(inp, decoded);

  autoencoder.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['acc'])
  return autoencoder




# Training one model #
Taken from https://github.com/agohr/deep_speck/blob/master/train_nets.py and slightly adapted.

In [ ]:
def cyclic_lr(num_epochs, high_lr, low_lr):
  res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr);
  return(res);

In [ ]:
bs=5000;

def train_autoenc(model, num_epochs, num_rounds, X_train, Y_train, X_eval, Y_eval):
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, restore_best_weights= True);
    lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));
    history = model.fit(X_train, Y_train, batch_size= bs, epochs=num_epochs,validation_data=(X_eval, Y_eval), callbacks=[lr, stop_early])
    return(model);

# Training the autoencoder and evaltuating the results multiple times #

In [ ]:
def multiple_trials(repetitions, num_rounds):
 
  accs = [];
  tprs = [];
  tnrs = [];
  for i in range(0, repetitions):
    X_train, Y_train = make_train_data(10**7, num_rounds);
    X_eval, Y_eval = make_train_data(10**6, num_rounds);
    initial_model = build_autoencoder();
    trained_model= train_autoenc(initial_model, 30, num_rounds, X_train, X_train, X_eval, X_eval);

    (acc, tpr, tnr) = multiple_evaluations(trained_model,5,num_rounds);
    accs.append(acc);
    tprs.append(tpr);
    tnrs.append(tnr);

  print("Acc: " + str(np.mean(accs)) + str(" +- ") + str(np.std(accs)) + str("\t") + 
        "Tpr:" + str(np.mean(tprs)) + str(" +- ") + str(np.std(tprs)) + str("\t") +
        "Tnr:" + str(np.mean(tnrs)) + str(" +- ") + str(np.std(tnrs)) + str("\t"));
        
  return(np.mean(accs), np.mean(tprs), np.mean(tnrs));

In [ ]:
multiple_trials(repetitions = 5, num_rounds =5) #num_rounds = 6, 7, 8 with an autoencoder with 1/2/3 layers (need to un/comment the lines in the autoencoder implementation)

# Now, train with a preprocessing step done with an encoder  #

# Train the autoencoder of choice # 

In [ ]:
num_rounds = 5;

X_train, Y_train = make_train_data(10**7, num_rounds);
X_eval, Y_eval = make_train_data(10**6, num_rounds);
    
initial_model = build_autoencoder();
trained_model= train_autoenc(initial_model, 30, num_rounds, X_train, X_train, X_eval, X_eval);


# Create encoder model and set its weights to the the pretrained weights from the encoder trained inside the autoencoder from above #

In [ ]:
inpe = Input(shape=(64,));
oute = encode(inpe); 
encoder_model = Model(inpe,oute)

encoder_model.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['acc']);

for l1,l2 in zip(encoder_model.layers[:18],trained_model.layers[0:18]):
    l1.set_weights(l2.get_weights())
    print(l1.name)

# Do not allow it the weights of the encoder to change #

In [ ]:
for layer in encoder_model.layers:
    layer.trainable = False

# Add the preprocessing step to the (reduced/depth-1/10) distinguisher - Pruned version #
Taken from https://github.com/agohr/deep_speck/blob/master/train_nets.py and slightly adapted.

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv1D,Conv2D, Input, Reshape, Permute, Add, Flatten, BatchNormalization, Activation, MaxPooling1D, Concatenate,Dropout, AveragePooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D, UpSampling1D
from keras.regularizers import l2, l1, l1_l2


def cyclic_lr(num_epochs, high_lr, low_lr):
  res = lambda i: low_lr + ((num_epochs-1) - i % num_epochs)/(num_epochs-1) * (high_lr - low_lr);
  return(res);

def make_checkpoint_best_worst(datei):
  res = ModelCheckpoint(datei, monitor='val_acc', save_best_only = True, save_weights_only=True );
  return(res);

def make_checkpoint_all(datei):
  res = ModelCheckpoint(datei, monitor='val_acc', save_best_only = False, save_weights_only=True);
  return(res);


wdir_curent_bw = "./current_bw/";
wdir_curent_bw_all= "./current_bw_all/";

bs = 5000;

def make_resnet( num_blocks=2, num_filters=32, num_outputs=1, d1=64, d2=64, word_size=16, ks=3,depth=5, reg_param=0.0001, final_activation='sigmoid'):
  #Input and preprocessing layers
  #Input and preprocessing layers

  #input is 10^7 samples of 64 bits each
  inp = Input(shape=(64,));
 
  x = encoder_model.call(inp); # encode the input

  
  
  conv0 = Conv1D(25, kernel_size=1, padding='same')(x);
  conv0 = BatchNormalization()(conv0);
  conv0 = Activation('relu')(conv0);

  conv1 = Conv1D(11, kernel_size=ks, padding='same')(conv0);
  conv1 = BatchNormalization()(conv1);
  conv1 = Activation('relu')(conv1);

  conv2 = Conv1D(7, kernel_size=ks, padding='same')(conv1);
  conv2 = BatchNormalization()(conv2);
  conv2 = Activation('relu')(conv2);

  flat = Flatten()(conv2);

  dense1 = Dense(18)(flat);
  dense1 = BatchNormalization()(dense1);
  dense1 = Activation('relu')(dense1);
  dense2 = Dense(28)(dense1);
  dense2 = BatchNormalization()(dense2);
  dense2 = Activation('relu')(dense2);
  out = Dense(1, activation='sigmoid')(dense2);
  model = Model(inputs=inp, outputs=out);

  return model

def model_builder(depth):

  model = make_resnet(depth=depth);
  model.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['acc']);
  return model;
  



def train_speck_distinguisher(model, num_epochs, num_rounds, X_train, Y_train, X_eval, Y_eval):
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, restore_best_weights= True);
    lr = LearningRateScheduler(cyclic_lr(10,0.002, 0.0001));
    history = model.fit(X_train, Y_train, batch_size= bs, epochs=num_epochs,validation_data=(X_eval, Y_eval), callbacks=[lr, stop_early])
    
    return(model);


In [ ]:
initial_modele = model_builder(depth=1); #modify the number of filters/neurons back to the original values found  https://github.com/agohr/deep_speck/blob/master/train_nets.py  and the depth to 10 to conduct the other experiments
trained_modele= train_speck_distinguisher(initial_modele, 30, num_rounds, X_train, Y_train, X_eval, Y_eval);